In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
df_heart = pd.read_csv('CleanHeart.csv')
df_heart.drop('Unnamed: 0',axis=1 , inplace=True)
df_heart.head()

,age,sex,resting_blood_pressure,cholestoral,fasting_blood_sugar,Max_heart_rate,exercise_induced_angina,oldpeak,target
0,52,1,125,212,0,168,0,1.0,0
1,53,1,140,203,1,155,1,3.1,0
2,70,1,145,174,0,125,1,2.6,0
3,61,1,148,203,0,161,0,0.0,0
4,62,0,138,294,1,106,0,1.9,0


### Improving Model (by Hyper parameters manually)

In [8]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

In [9]:
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

## Hyperparamaeters 
## 1 Manually
## 2 Randomly with RandomSearchCV
## 3 Exhaustively GridSearchCV

we split data into ,test, train, validate data , 
hyperparmets are used on validate data

#### 1. we split data into ,test, train, validate data , 
#### hyperparmets are used on validate data

#### 2. The parameters we are going to use
max_depth
max_features 
min_samples_split
min_sample_leaf


In [ ]:
def evaluate_hyper_metrics(y_true,y_predicted):
    """This Function prints Returns Evaluation Matrices"""

    accuracy  = accuracy_score(y_true,y_predicted)
    precision  = precision_score(y_true,y_predicted) 
    recall  = recall_score(y_true,y_predicted) 
    f1  = f1_score(y_true,y_predicted) 

    metrics_dict = {'Accuracy': accuracy,
                   'Precision' : precision,
                   'Recall' : recall, 
                   'f1' : f1}
    
    print(f'Accuracy Score : {round(accuracy_score(y_true,y_predicted),2)}')
    print(f'Precision score : {round(precision_score(y_true,y_predicted),2)}')
    print(f'Recall Score : {round(recall_score(y_true,y_predicted),2)}')
    print(f'f1 Score : {round(f1_score(y_true,y_predicted),2)}')

    return metrics_dict

# Task 1

In [3]:
# splitting test into test ,train, validate 
mixed_heart = df_heart.sample(frac=1)

#split x , y
x = df_heart.drop('target',axis=1)
y = df_heart['target']

# splitting data into three parts : Train: 70% , Test : 15% , Validate : 15%
train_split = round(0.7*len(mixed_heart))
validate_split = round(train_split + 0.15*len(mixed_heart))

x_train,y_train = x[:train_split],y[:train_split]
x_valid,y_valid = x[train_split:validate_split],y[train_split:validate_split]
x_test,y_test = x[validate_split:],y[validate_split:]

len(mixed_heart)

302

In [12]:
len(x_train),len(x_valid),len(x_test)

(211, 45, 46)

In [13]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
np.random.seed(7)

In [14]:
# first Result
np.random.seed(0)
clf = RandomForestClassifier()
model = clf.fit(x_train,y_train)
#baseline prediction
y_predicted = clf.predict(x_valid)
# Evaluating the model 
first_result = evaluate_hyper_metrics(y_valid,y_predicted)  # funstion is build above


Accuracy Score : 0.69
Precision score : 0.74
Recall Score : 0.61
f1 Score : 0.67


In [15]:
# Second Result
np.random.seed(0)
clf2 = RandomForestClassifier(n_estimators=100)
model2 = clf2.fit(x_train,y_train)
#baseline prediction
y_predicted = clf2.predict(x_valid)
# Evaluating the model 
second_result = evaluate_hyper_metrics(y_valid,y_predicted)  # funstion is build above


Accuracy Score : 0.69
Precision score : 0.74
Recall Score : 0.61
f1 Score : 0.67


In [20]:
# Third Result
np.random.seed(0)
clf3 = RandomForestClassifier(max_depth=10)
model3 = clf3.fit(x_train,y_train)
#baseline prediction
y_predicted = clf3.predict(x_valid)
# Evaluating the model 
Third_result = evaluate_hyper_metrics(y_valid,y_predicted)  # funstion is build above


Accuracy Score : 0.89
Precision score : 0.95
Recall Score : 0.83
f1 Score : 0.88


### Tuning Hyperparameters usnig Randomized Search CV (RSCV)

In [21]:
'max_depth' : [None , 5, 10,20,30]
'n_estimators' : [10 , 100 ,200 , 500, 1000, 12000]
'min_samples_split' : [2,4,6]
'min_samples_leaf' : [1,2,4]
'max_features': ['sqrt', 'log2', None]   #Both 'auto' and 'sort' are invalid in the latest versions of scikit-learn.
'n_jobs'

SyntaxError: illegal target for annotation (1029783665.py, line 1)

In [23]:
from sklearn.model_selection import RandomizedSearchCV
h_paramas = {'max_depth' : [None , 5, 10,20,30], 
             'n_estimators' : [10 , 100 ,200 , 500, 1000, 12000], 
             'min_samples_split' : [2,4,6],
             'min_samples_leaf' : [1,2,4], 
             'max_features': ['sqrt', 'log2', None]}

#split data into target and features 
x = mixed_heart.drop('target',axis=1)
y = mixed_heart['target']

#split into train , test 
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

clf = RandomForestClassifier(n_jobs=1)

rscv_clf = RandomizedSearchCV(clf , h_paramas , n_iter=10, cv=5)
# it will take aprox 5 min to run

In [24]:
rscv_clf.fit(x_train,y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=1),
                   param_distributions={'max_depth': [None, 5, 10, 20, 30],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 4, 6],
                                        'n_estimators': [10, 100, 200, 500,
                                                         1000, 12000]})

In [26]:
rscv_clf.best_params_

{'n_estimators': 12000,
 'min_samples_split': 6,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 5}

In [29]:
rscv_predicted = rscv_clf.predict(x_test)
# evaluate the prediction
fourth_result = evaluate_hyper_metrics(y_valid, y_predicted)

Accuracy Score : 0.89
Precision score : 0.95
Recall Score : 0.83
f1 Score : 0.88


### Tuning the Hyperparameters using grid search validation

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
h_paramas = {'max_depth' : [None , 5, 10], 
             'n_estimators' : [500,1000, 12000], 
             'min_samples_split' : [2,4,6],
             'min_samples_leaf' : [2,4], 
             'max_features': ['sqrt', 'log2']}

#split data into target and features 
x = mixed_heart.drop('target',axis=1)
y = mixed_heart['target']

#split into train , test 
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

clf = RandomForestClassifier(n_jobs=1)
gsv_clf = GridSearchCV(clf , h_paramas , cv=5, verbose=2)

#fitting
gsv_clf.fit(x_train,y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=500; total time=   3.5s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=500; total time=   4.8s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=500; total time=   4.5s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=500; total time=   2.5s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=500; total time=   3.4s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   8.3s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   5.5s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split

In [ ]:
gsv_clf.best_params_

In [ ]:
gsv_clf_y_predicted = gsv_clf.predict(x_test)
#evaluate the metrics 
fifth_result = evaluate_hyper_metrics(y_test,gsv_clf_y_predicted)